# Model serving

In [1]:
import ray
from ray import serve

In [2]:
!python --version

Python 3.8.13


In [3]:
ray_head = "ray-head"
ray.init(address=f'ray://{ray_head}:10001')

Python version:,3.8.13
Ray version:,2.2.0
Dashboard:,http://172.20.0.5:8265


In [4]:
serve.start(detached=False, http_options={'host':"0.0.0.0", 'port':5010})
serve.start(detached=True, http_options={'host':"0.0.0.0", 'port':5010})


(ServeController pid=807) INFO 2023-03-21 08:18:22,812 controller 807 http_state.py:129 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:UGoFTJ:SERVE_PROXY_ACTOR-6295723a3edf21836f951a4d8d2a0a76ea90155e233d6051c6ab30de' on node '6295723a3edf21836f951a4d8d2a0a76ea90155e233d6051c6ab30de' listening on '0.0.0.0:5010'


(HTTPProxyActor pid=870) INFO:     Started server process [870]


In [ ]:
import requests

try:
    response = requests.get(f"http://{ray_head}:5010")
    #response.raise_for_status()
    print("Modelo esta desplegado")
    #serve.shutdown()
except requests.exceptions.RequestException as e:
    print("Modelo no esta desplegado. ", e)
    print("Desplegando el modelo:")
    serve.start(detached=True, http_options={'host':"0.0.0.0", 'port':5010}) # En produccion, ver bien el puerto

## Server version 2: Ray + FastAPI

In [5]:
import fastapi

fastapi.__version__

'0.95.0'

In [10]:
import mlflow
import pandas as pd

from fastapi import FastAPI, Query
from pydantic import BaseModel#, Field
from fastapi.responses import HTMLResponse
from fastapi.responses import RedirectResponse


class Input(BaseModel):
    specimen_number: float = Query(..., gt=0, example=1)
    eccentricity: float = Query(..., gt=0, example=0.86224)
    aspect_ratio: float = Query(..., gt=0, example=2.0735)
    elongation: float = Query(..., gt=0, example=0.52269)
    solidity: float = Query(..., gt=0, example=0.98686)
    stochastic_convexity: float = Query(..., gt=0, example=0.99474)
    isoperimetric_factor: float = Query(..., gt=0, example=0.70529)
    maximal_indentation_depth: float = Query(..., gt=0, example=0.010097)
    lobedness: float = Query(..., gt=0, example=0.018554)
    average_intensity: float = Query(..., gt=0, example=0.041404)
    average_contrast: float = Query(..., gt=0, example=0.12163)
    smoothness: float = Query(..., gt=0, example=0.014579)
    third_moment: float = Query(..., gt=0, example=0.0048689)
    uniformity: float = Query(..., gt=0, example=0.00027608)
    entropy: float = Query(..., gt=0, example=0.9458)


app = FastAPI(title='Predictor API',
              description='Pipeline online inference')


@serve.deployment()
@serve.ingress(app)
# @serve.deployment()
class LeafDeployment:
    def __init__(self):
        model_name = "extratree"
        model_stage = "Production"
        self.predictor = mlflow.sklearn.load_model(
                                model_uri=f"models:/{model_name}/{model_stage}")
        self.predictor2 = mlflow.sklearn.load_model(
                                model_uri=f"models:/{model_name}/6")
        self.predictor3 = mlflow.sklearn.load_model(
                                model_uri=f"models:/{model_name}/7")

    def preprocessing(self, df):
        with mlflow.start_run(run_name='preprocessing') as mlrun:
            # Some preprocessing steps here
            # df = pd.read_csv(filename)
            df_cleaned = df.loc[:, df.columns != 'specimen_number'].copy()
            df_cleaned[df_cleaned.columns] = df_cleaned[df_cleaned.columns].astype(float)
            df_cleaned.to_csv('preprocessed_data.csv', index=False)
            mlflow.log_artifact('preprocessed_data.csv')

            # logging
            mlflow.log_param(key='n_samples', value=len(df_cleaned))
            mlflow.log_param(key='n_features', value=len(df_cleaned.columns))

            return df_cleaned

    @app.get('/', include_in_schema=False)
    async def docs_redirect(self):
        return RedirectResponse(url='/docs')

    @app.post("/predict",
         tags=['Predictor 1 por default'],
         summary="Usa el modelo 1 para la predicción")
    def call(self, request: Input):
    # async def __call__(self, request):
        # data = await request.json()
        data = request.__dict__
        data = pd.json_normalize(data)
        preprocessed = self.preprocessing(data)
        result = self.predictor.predict(preprocessed)
        return result
    
    @app.post("/model2",
         tags=['Predictor 2'],
         summary="Usa el modelo 2 para la predicción")
    def call(self, request: Input):
    # async def __call__(self, request):
        # data = await request.json()
        data = request.__dict__
        data = pd.json_normalize(data)
        preprocessed = self.preprocessing(data)
        result = self.predictor2.predict(preprocessed)
        return result
    
    @app.post("/model3",
         tags=['Predictor 3'],
         summary="Usa el modelo 3 para la predicción")
    def call(self, request: Input):

        # data = await request.json()
        data = request.__dict__
        data = pd.json_normalize(data)
        preprocessed = self.preprocessing(data)
        result = self.predictor3.predict(preprocessed)
        return result
    
    @app.post("/model_voting",
         tags=['Voting method'],
         summary="Calcula la moda de las predicciones")
    def call(self, request: Input):
        from scipy import stats
        
        data = request.__dict__
        data = pd.json_normalize(data)
        preprocessed = self.preprocessing(data)
        predicts = [self.predictor.predict(preprocessed),
                   self.predictor2.predict(preprocessed),
                   self.predictor3.predict(preprocessed)]
        result = stats.mode(predicts, keepdims=True).mode[0]
        return result
    
    @app.post("/checker_integrity_model",
         tags=['Checker Integrity + model'],
         summary="Ejecuta el checker de integridad y luego el modelo")
    def call(self, request: Input):
        from scipy import stats
        
        data = request.__dict__
        if data['entropy'] >= 10:
            return -1
        data = pd.json_normalize(data)
        preprocessed = self.preprocessing(data)
        result = self.predictor3.predict(preprocessed)
        return result
        
# LeafDeployment.deploy()
serve.run(LeafDeployment.bind())

The new client HTTP config differs from the existing one in the following fields: ['host', 'port', 'location']. The new HTTP config is ignored.
(ServeController pid=807) INFO 2023-03-21 08:29:02,806 controller 807 deployment_state.py:1214 - Stopping 1 replicas of deployment 'LeafDeployment' with outdated versions.
(ServeController pid=807) INFO 2023-03-21 08:29:04,963 controller 807 deployment_state.py:1310 - Adding 1 replica to deployment 'LeafDeployment'.
(ServeReplica:LeafDeployment pid=1256) Exception in 'lifespan' protocol
(ServeReplica:LeafDeployment pid=1256) Traceback (most recent call last):
(ServeReplica:LeafDeployment pid=1256)   File "/home/ray/anaconda3/lib/python3.8/site-packages/uvicorn/lifespan/on.py", line 86, in main
(ServeReplica:LeafDeployment pid=1256)     await app(scope, self.receive, self.send)
(ServeReplica:LeafDeployment pid=1256)   File "/home/ray/anaconda3/lib/python3.8/site-packages/uvicorn/middleware/proxy_headers.py", line 78, in __call__
(ServeReplica:Le

RayServeSyncHandle(deployment='LeafDeployment')

# Cliente

In [11]:
import pandas as pd
import requests

filename = "artifacts/X_inference.csv"
df = pd.read_csv(filename)
# request = df.head(1).to_dict('list')
request = df.head(1).to_dict('records')
request

[{'specimen_number': 1,
  'eccentricity': 0.86224,
  'aspect_ratio': 2.0735,
  'elongation': 0.52269,
  'solidity': 0.98686,
  'stochastic_convexity': 0.99474,
  'isoperimetric_factor': 0.70529,
  'maximal_indentation_depth': 0.010097,
  'lobedness': 0.018554,
  'average_intensity': 0.041404,
  'average_contrast': 0.12163,
  'smoothness': 0.014579,
  'third_moment': 0.0048689,
  'uniformity': 0.00027608,
  'entropy': 0.9458}]

## Llamar al modelo 1, por default

In [12]:
%%time

# response = requests.post("http://0.0.0.0:5010/predict", json=request[0])
response = requests.post("http://ray-head:5010/predict", json=request[0])
# response = requests.post("http://ray-head:5010/LeafDeployment", json=request)
result = response.json()[0]
result

(ServeReplica:LeafDeployment pid=1256) ERROR 2023-03-21 08:29:26,842 LeafDeployment LeafDeployment#lKfgOl replica.py:461 - Request failed due to TypeError:
(ServeReplica:LeafDeployment pid=1256) Traceback (most recent call last):
(ServeReplica:LeafDeployment pid=1256)   File "/home/ray/anaconda3/lib/python3.8/site-packages/ray/serve/_private/replica.py", line 443, in invoke_single
(ServeReplica:LeafDeployment pid=1256)     result = await method_to_call(*args, **kwargs)
(ServeReplica:LeafDeployment pid=1256)   File "/opt/conda/lib/python3.8/site-packages/ray/serve/api.py", line 222, in __call__
(ServeReplica:LeafDeployment pid=1256)   File "/home/ray/anaconda3/lib/python3.8/site-packages/fastapi/applications.py", line 270, in __call__
(ServeReplica:LeafDeployment pid=1256)     await super().__call__(scope, receive, send)
(ServeReplica:LeafDeployment pid=1256)   File "/home/ray/anaconda3/lib/python3.8/site-packages/starlette/applications.py", line 124, in __call__
(ServeReplica:LeafDeplo

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

(HTTPProxyActor pid=870) INFO 2023-03-21 08:29:26,845 http_proxy 172.20.0.5 http_proxy.py:361 - POST / 500 6.9ms
(HTTPProxyActor pid=870) Task exception was never retrieved
(HTTPProxyActor pid=870) future: <Task finished name='Task-24' coro=<_wrap_awaitable() done, defined at /home/ray/anaconda3/lib/python3.8/asyncio/tasks.py:688> exception=RayTaskError(TypeError)(TypeError("'NoneType' object is not callable"))>
(HTTPProxyActor pid=870) Traceback (most recent call last):
(HTTPProxyActor pid=870)   File "/home/ray/anaconda3/lib/python3.8/asyncio/tasks.py", line 695, in _wrap_awaitable
(HTTPProxyActor pid=870)     return (yield from awaitable.__await__())
(HTTPProxyActor pid=870) ray.exceptions.RayTaskError(TypeError): ray::ServeReplica:LeafDeployment.handle_request() (pid=1256, ip=172.20.0.5)
(HTTPProxyActor pid=870)   File "/home/ray/anaconda3/lib/python3.8/site-packages/ray/serve/_private/utils.py", line 238, in wrap_to_ray_error
(HTTPProxyActor pid=870)     raise exception
(HTTPProxy

## Llamar al modelo 2

In [9]:
%%time

response = requests.post("http://ray-head:5010/model2", json=request[0])
# response = requests.post("http://ray-head:5010/LeafDeployment", json=request)
result = response.json()[0]
result

CPU times: user 9.97 ms, sys: 699 µs, total: 10.7 ms
Wall time: 98.7 ms


32

(HTTPProxyActor pid=341) INFO 2022-11-08 10:19:24,620 http_proxy 172.22.0.4 http_proxy.py:315 - POST / 200 94.8ms
(ServeReplica:LeafDeployment pid=757) INFO 2022-11-08 10:19:24,618 LeafDeployment LeafDeployment#tamUxz replica.py:505 - HANDLE __call__ OK 90.2ms


## Llamar al model_voting

In [22]:
%%time

response = requests.post("http://ray-head:5010/model_voting", json=request[0])
result = response.json()[0]
result

CPU times: user 5.25 ms, sys: 9.15 ms, total: 14.4 ms
Wall time: 167 ms


32

(HTTPProxyActor pid=341) INFO 2022-11-08 11:42:53,742 http_proxy 172.22.0.4 http_proxy.py:315 - POST / 200 162.7ms
(ServeReplica:LeafDeployment pid=2305) INFO 2022-11-08 11:42:53,740 LeafDeployment LeafDeployment#FytfAS replica.py:505 - HANDLE __call__ OK 159.1ms
(HTTPProxyActor pid=341) INFO 2022-11-08 11:43:41,085 http_proxy 172.22.0.4 http_proxy.py:315 - POST / 200 112.0ms
(ServeReplica:LeafDeployment pid=2305) INFO 2022-11-08 11:43:41,083 LeafDeployment LeafDeployment#FytfAS replica.py:505 - HANDLE __call__ OK 107.9ms
